# Imports

In [1]:
# If you have not installed `wiscs` locally, run this cell
!pip install git+https://github.com/w-decker/wiscs.git --quiet # REQUIRED FOR THIS NOTEBOOK
!pip install git+https://github.com/w-decker/rinterface.git --quiet # REQUIRED FOR THIS NOTEBOOK

In [2]:
# always run this cell
import wiscs
from wiscs.simulate import DataGenerator
from wiscs.utils import make_tasks
from wiscs.formula import Formula

import rinterface.rinterface as R
from rinterface.utils import to_r

from src.utils import fmt_script

import numpy as np

# Generating data
Data are generated using the [`wiscs`](https://github.com/w-decker/wiscs) framework. For now, arbitrary variance parameters and design choices are set to help "build" the linear model. Once a final model has been chosen, additional power analyses can be run to determine the exact experimental design criteria.

>Please see [generate_data.ipynb](/notebooks/generate_data.ipynb) for information on how to use the `wiscs` module. 

In [19]:
task = make_tasks(low=100, high=200, n=5)
params = {'word.perceptual': 100, 'image.perceptual': 95, 'word.conceptual': 100, 'image.conceptual': 100, 'word.task': task, 'image.task': task,
        # noise parameters     
        'sd.item': None,     'sd.question': None,    'sd.subject': 20,       "sd.modality": None, "sd.error": 50, "sd.re_formula": "(1|subject)",
        # correlations among random effects    
        "corr.subject": np.array([[1]]),
        # design parameters
        'n.subject': 10, 'n.question': 5, 'n.item': 10
}
wiscs.set_params(params)

# generate data
DG = DataGenerator()
df = DG.fit_transform(seed=2025).to_pandas()

Params set successfully


# Evaluate in R with [`rinterface`](https://github.com/w-decker/rinterface)

I have built a small interface between Python and R. Check out the repo [here](https://github.com/w-decker/rinterface). Essentially, it takes in a multiline string containing an R-valid script and runs that as a subprocess using the `Rscript` command. There's a lot more to it than that (check out the repo for examples and additional functionality), but that's the jist. For the sake of brevity, I've condensed the script we will be regularly running inside a function [`src.fmt_script()`](/notebooks/src/utils.py) (see line 144). This will keep the notebook cleaner. A few things to note on `fmt_script()`:

### What's inside `fmt_script()`?
1. Imports necessary packages, including `lme4` and `lmerTest`. 
2. Factorizes categorical variables (this is hardcoded in because we already know what variables are which)
3. Establishes treatment codes for categorical variables. You can print the script to see more details.
4. Runs the models and prints the summary of each
5. Runs `anova` on the two models

### What do _you_ need to give `fmt_script()`?
1. The R formulas for the shared and separate model as a string
2. The pandas dataframe containing the data

### What if you want to add some more code?
You can optionally specify a list of strings containing new lines of code. Each element in your list will be a new line. These are then added right before the model is run: 

```python
fmt_script(shared_f=..., separate_f=..., df=df, add=['cat("hello world")'])
```

In [4]:
re_formula = Formula("(1 | subject)")
script = fmt_script(shared_re=re_formula, df=df)
R(script)


 SHARED model summary
Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  method [lmerModLmerTest]
Formula: rt ~ modality + question + (1 | subject)
   Data: df
Control: lmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
 10634.4  10673.7  -5309.2  10618.4      992 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.2610 -0.6799 -0.0003  0.6654  3.0704 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept)  438.9   20.95   
 Residual             2323.0   48.20   
Number of obs: 1000, groups:  subject, 10

Fixed effects:
            Estimate Std. Error      df t value Pr(>|t|)    
(Intercept)  354.886      7.450  14.420  47.636  < 2e-16 ***
modality1     -3.305      3.048 990.000  -1.084    0.278    
question1     20.841      4.820 990.000   4.324 1.69e-05 ***
question2    -29.725      4.820 990.000  -6.167 1.01e-09 ***
question3     43.810      4.820 990.000   9.090  < 2e-16 ***
question4    -

#### Using `wiscs.formula.Formula`

`wiscs` has a built in class, `Formula`, that's used to handle the random effects formula internally. However, it's accessible and handy for instances like this, where we are repeatedly modifying our random effects structure. 

```python
from wiscs.formula import Formula

f = Formula("(1 + A | B) + (1 | C)")
f
```
```plain
>>> (1 + A | B) + (1 | C)
```
```python
for i in f:
    print(i)
```
```plain
>>>(1 + A | B)
(1 | C)
```
```python
len(f)
```
```plain
>>> 2
```
```python
f + "(1 | D)"
```
```plain
>>> (1 + A | B) + (1 | C) + (1 | D)
```
```python
f - "(1 | C)"
```
```plain
>>>(1 + A | B) + (1 | D)
```

# What does the rest of this notebook look like?

Much of this notebook is evaluating different values specified in the data generation process and different formula's to specify a linear model. The point is to define a conceptually sound model that maximally aligns with the hypotheses laid out in the project. As such, the two previous codeblocks will be repeatedly run (in new cells) below. So much of what's written below is similar to you've already seen. All that's changed are values given to the data generator and the linear models.

## Starting small: 1 random effect

Here, we are modelling `subject` as a random effect. Specifically, we are giving subjects random _**intercepts**_. This allows subjects to vary in their mean reaction time inside the model. 

In [5]:
script = fmt_script(shared_re=re_formula, df=df)
R(script)


 SHARED model summary
Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  method [lmerModLmerTest]
Formula: rt ~ modality + question + (1 | subject)
   Data: df
Control: lmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
 10634.4  10673.7  -5309.2  10618.4      992 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.2610 -0.6799 -0.0003  0.6654  3.0704 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept)  438.9   20.95   
 Residual             2323.0   48.20   
Number of obs: 1000, groups:  subject, 10

Fixed effects:
            Estimate Std. Error      df t value Pr(>|t|)    
(Intercept)  354.886      7.450  14.420  47.636  < 2e-16 ***
modality1     -3.305      3.048 990.000  -1.084    0.278    
question1     20.841      4.820 990.000   4.324 1.69e-05 ***
question2    -29.725      4.820 990.000  -6.167 1.01e-09 ***
question3     43.810      4.820 990.000   9.090  < 2e-16 ***
question4    -

As you can see, this model does fairly well at recovering the standard deviations we've set in the simulation.

| **Variable** | **Given** | **Estimated** |
| ------------ | --------- | ------------- |
| Subject | 20 | 20.949 |
| Residual | 50 | 48.198 |

Our shared model performed as expected relative to the separate model:

```plain
 ANOVA
Data: df
Models:
shared: rt ~ modality + question + (1 | subject)
separate: rt ~ modality * question + (1 | subject)
         npar   AIC   BIC  logLik deviance  Chisq Df Pr(>Chisq)
shared      8 10634 10674 -5309.2    10618                     
separate   12 10640 10698 -5307.8    10616 2.8351  4     0.5858
```

There were no convergence issues. The model we've specified reliably parses the variance in our data. 

# Adding another random effect

Let's add another random effect: `item`. Here, we will simulate an random _**intercept**_ for item. This means that reaction times can vary by item.

>Note that we did not have any values set in our original data setup. We will need to update the `DataGenerator` with a value for `item` denoting the standard deviation to be used as the random intercept, as well as a correlation matrix and an updated random effects formula.

In [20]:
# update random effects
re_formula = Formula("(1 | subject) + (1 | item)")
# update data generator
DG.fit_transform({'sd.item':30, 'sd.re_formula':str(re_formula), 'corr.item':np.array([[1]])}, seed=2025)
# convert to Pandas DataFrame
df = DG.to_pandas()

In [7]:
script = fmt_script(shared_re=re_formula,df=df)
R(script)


 SHARED model summary
Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  method [lmerModLmerTest]
Formula: rt ~ modality + question + (1 | subject) + (1 | item)
   Data: df
Control: lmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
 10670.1  10714.3  -5326.1  10652.1      991 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-3.15571 -0.69088 -0.01792  0.66979  3.09151 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept)  394.2   19.85   
 item     (Intercept)  508.3   22.54   
 Residual             2336.7   48.34   
Number of obs: 1000, groups:  subject, 10; item, 10

Fixed effects:
            Estimate Std. Error      df t value Pr(>|t|)    
(Intercept)  344.198     10.096  21.641  34.093  < 2e-16 ***
modality1     -3.300      3.057 980.949  -1.080 0.280623    
question1     18.275      4.834 980.949   3.781 0.000166 ***
question2    -33.064      4.834 980.949  -6.840 1.39e-11 ***
qu

Okay great. So far no serious problems. Notice however, that we don't really do a good job of estimating our item parameter. This could be due to the small number of examples in the model (10). Let's just see what happens to our estimate for items when we add a few more examples into the model.

In [21]:
# update data generator
DG.fit_transform({'n.item':20}, seed=2025)
# convert to Pandas DataFrame
df = DG.to_pandas()
script = fmt_script(shared_re=re_formula,df=df)
R(script)


 SHARED model summary
Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  method [lmerModLmerTest]
Formula: rt ~ modality + question + (1 | subject) + (1 | item)
   Data: df
Control: lmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
 21438.8  21489.2 -10710.4  21420.8     1991 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.2677 -0.6867 -0.0096  0.6638  2.9786 

Random effects:
 Groups   Name        Variance Std.Dev.
 item     (Intercept)  619.0   24.88   
 subject  (Intercept)  479.2   21.89   
 Residual             2497.4   49.97   
Number of obs: 2000, groups:  item, 20; subject, 10

Fixed effects:
            Estimate Std. Error       df t value Pr(>|t|)    
(Intercept)  329.416      9.226   23.961  35.706  < 2e-16 ***
modality1     -3.108      2.235 1970.969  -1.391    0.165    
question1     47.028      3.534 1970.969  13.308  < 2e-16 ***
question2    -29.885      3.534 1970.969  -8.457  < 2e-16 ***
questio

As you can see, we have a better estimate for the variance that we've injected into the data.

# Let's keep going
Okay so far no major problems. We have specified two random intercepts (`subject` and `item`), which means that we expect different means for each participant and item. Let's cap our intercepts at these two. But before we finalize the model, let's back-track. We also want some random slopes too. For example, I expect that mean reaction times might deviate at each level of question. So we have a random slope of `question` grouped by `subject`, for example. Let's try this. 

In [23]:
# update random effects
re_formula = Formula("(1 + question | subject)")

In [24]:
# update data generator
sub_corr = np.eye(5)
# sub_corr = np.eye(6)
DG.fit_transform({'sd.question':[10, 12, 15, 18], 'sd.re_formula':str(re_formula), 'corr.subject':sub_corr}, seed=2025)
# convert to Pandas DataFrame
df = DG.to_pandas()

In [11]:
R(fmt_script(shared_re=re_formula, df=df))

boundary (singular) fit: see help('isSingular')
boundary (singular) fit: see help('isSingular')



 SHARED model summary
Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  method [lmerModLmerTest]
Formula: rt ~ modality + question + (1 + question | subject)
   Data: df
Control: lmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
 21408.0  21531.2 -10682.0  21364.0     1978 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.5077 -0.6762  0.0076  0.6449  3.0853 

Random effects:
 Groups   Name        Variance Std.Dev. Corr                   
 subject  (Intercept)  850.94  29.171                          
          question1     71.09   8.431   -0.39                  
          question2     80.47   8.971    0.09  0.05            
          question3     86.53   9.302   -0.50  0.76 -0.42      
          question4    287.56  16.958   -0.46  0.14 -0.89  0.47
 Residual             2465.39  49.653                          
Number of obs: 2000, groups:  subject, 10

Fixed effects:
            Estimate Std. Error       df 

Okay we have a singlur boundary warning. This means that the variance estimates are close to zero. Technically, singular models are not invalid, but they do correspond to overfitting and are at an increased likelihood for convergence issues. Let's start by increasing the number of subject from 10 to 100.

In [25]:
DG.fit_transform({'n.subject':100}, seed=2025)
# convert to Pandas DataFrame
df = DG.to_pandas()

In [15]:
R(fmt_script(df=df, shared_re=re_formula, VarCorr_only=True))


 Variance components (Shared Model)
 Groups   Name        Std.Dev. Corr                       
 subject  (Intercept) 17.470                              
          question1    9.341    0.158                     
          question2   11.396    0.106  0.117              
          question3   14.606    0.349 -0.221  0.056       
          question4   18.544   -0.075  0.021  0.094 -0.144
 Residual             50.032                              

 Variance components (Separate Model)
 Groups   Name        Std.Dev. Corr                       
 subject  (Intercept) 17.471                              
          question1    9.342    0.158                     
          question2   11.397    0.106  0.117              
          question3   14.606    0.349 -0.221  0.056       
          question4   18.545   -0.075  0.021  0.094 -0.144
 Residual             50.028                              


No issues now!

# Increasing complexity

Let's add `item` back in as an intercept.

In [26]:
re_formula = Formula("(1 + question | subject) + (1 |item)")

In [27]:
DG.fit_transform({'sd.item':30, 'sd.re_formula':str(re_formula)}, seed=2025)
# convert to Pandas DataFrame
df = DG.to_pandas()

In [26]:
R(fmt_script(df=df, shared_re=re_formula))


 SHARED model summary
Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  method [lmerModLmerTest]
Formula: rt ~ modality + question + (1 + question | subject) + (1 | item)
   Data: df
Control: lmerControl(optimizer = "bobyqa")

      AIC       BIC    logLik  deviance  df.resid 
 214224.1  214405.9 -107089.1  214178.1     19977 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.0624 -0.6813  0.0004  0.6757  3.6652 

Random effects:
 Groups   Name        Variance Std.Dev. Corr                   
 subject  (Intercept)  356.32  18.876                          
          question1    154.91  12.446   -0.10                  
          question2     97.98   9.899    0.13  0.14            
          question3    202.05  14.214    0.16 -0.10  0.22      
          question4    341.34  18.475   -0.10  0.04 -0.09 -0.22
 item     (Intercept)  604.16  24.580                          
 Residual             2504.79  50.048                          
Number of ob

So far so good! But check out the `lme4::anova()` output

```
 ANOVA
Data: df
Models:
shared: rt ~ modality + question + (1 + question | subject) + (1 | item)
separate: rt ~ modality * question + (1 + question | subject) + (1 | item)
         npar    AIC    BIC  logLik deviance  Chisq Df Pr(>Chisq)
shared     23 214224 214406 -107089   214178                     
separate   27 214225 214438 -107085   214171 7.3102  4     0.1204
```

Looks like we are approaching significant differences between our separate and shared model, despite the structure of the data aligning conceptually with the shared model. Let's see if adding more subject helps.

In [28]:
DG.fit_transform({'n.subject':200, 'sd.re_formula':str(re_formula)}, seed=2025)
# convert to Pandas DataFrame
df = DG.to_pandas()

In [28]:
R(fmt_script(df=df, shared_re=re_formula))


 SHARED model summary
Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  method [lmerModLmerTest]
Formula: rt ~ modality + question + (1 + question | subject) + (1 | item)
   Data: df
Control: lmerControl(optimizer = "bobyqa")

      AIC       BIC    logLik  deviance  df.resid 
 428472.8  428670.5 -214213.4  428426.8     39977 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.0396 -0.6695  0.0004  0.6673  3.6907 

Random effects:
 Groups   Name        Variance Std.Dev. Corr                   
 subject  (Intercept)  313.47  17.705                          
          question1     85.66   9.255   -0.04                  
          question2    170.60  13.061   -0.10 -0.04            
          question3    215.79  14.690    0.03  0.03  0.15      
          question4    304.73  17.457   -0.02 -0.12  0.04  0.05
 item     (Intercept)  813.81  28.527                          
 Residual             2516.57  50.165                          
Number of ob

Looks better! Let's try adding in a slope for `question` that's grouped by `item`.

In [29]:
re_formula = Formula("(1 + question | subject) + (1 + question | item)")

In [30]:
DG.fit_transform({'corr.item':np.eye(5), 'sd.re_formula':str(re_formula)}, seed=2025)
# convert to Pandas DataFrame
df = DG.to_pandas()

In [32]:
R(fmt_script(df=df, shared_re=re_formula))


 SHARED model summary
Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  method [lmerModLmerTest]
Formula: rt ~ modality + question + (1 + question | subject) + (1 + question |  
    item)
   Data: df
Control: lmerControl(optimizer = "bobyqa")

      AIC       BIC    logLik  deviance  df.resid 
 428761.6  429079.7 -214343.8  428687.6     39963 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9533 -0.6724  0.0010  0.6714  3.6801 

Random effects:
 Groups   Name        Variance Std.Dev. Corr                   
 subject  (Intercept)  281.6   16.78                           
          question1    114.2   10.69    -0.01                  
          question2    133.4   11.55     0.14 -0.03            
          question3    248.5   15.76     0.20  0.00 -0.08      
          question4    336.6   18.35    -0.03  0.22  0.10  0.11
 item     (Intercept)  394.1   19.85                           
          question1    125.2   11.19    -0.08              

Great! That took a little longer to run (and I was nervous about convergence issues) but it looks pretty good! The model does okay at estimating our artifical variance parameters and there are no singular fits. Let's try this again with a few more subjects. 

In [31]:
DG.fit_transform({'n.subject':300}, seed=2025)
# convert to Pandas DataFrame
df = DG.to_pandas()

In [35]:
R(fmt_script(df=df, shared_re=re_formula, VarCorr_only=True))


 Variance components (Shared Model)
 Groups   Name        Std.Dev. Corr                       
 subject  (Intercept) 18.1757                             
          question1    9.6575   0.132                     
          question2   13.0629  -0.020 -0.062              
          question3   15.1669   0.037 -0.055 -0.009       
          question4   17.0728  -0.016  0.040 -0.023  0.037
 item     (Intercept) 37.6182                             
          question1   11.3916  -0.304                     
          question2    9.2925   0.119 -0.172              
          question3   11.2813   0.246  0.201 -0.571       
          question4   16.5750  -0.140  0.382 -0.085 -0.041
 Residual             50.1992                             

 Variance components (Separate Model)
 Groups   Name        Std.Dev. Corr                       
 subject  (Intercept) 18.1758                             
          question1    9.6578   0.132                     
          question2   13.0632  -0.020 -

Better!

# Increasing complexity...again

Let's try to add a random slope for `modality` grouped by `subject`

In [32]:
re_formula = Formula("(1 + question + modality | subject) + (1 + question | item)")

In [33]:
DG.fit_transform({'corr.subject':np.eye(6),'sd.modality':10, 'sd.re_formula':str(re_formula)}, seed=2025)
# convert to Pandas DataFrame
df = DG.to_pandas()

In [38]:
R(fmt_script(df=df, shared_re=re_formula))

Warning message:
In commonArgs(par, fn, control, environment()) :
  maxfun < 10 * length(par)^2 is not recommended.



 SHARED model summary
Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  method [lmerModLmerTest]
Formula: rt ~ modality + question + (1 + question + modality | subject) +  
    (1 + question | item)
   Data: df
Control: lmerControl(optimizer = "bobyqa")

      AIC       BIC    logLik  deviance  df.resid 
 643390.6  643777.7 -321652.3  643304.6     59957 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.0683 -0.6701  0.0027  0.6676  4.4536 

Random effects:
 Groups   Name          Variance Std.Dev. Corr                         
 subject  (Intercept)    431.34  20.769                                
          question1      109.60  10.469   -0.12                        
          question2      157.31  12.542   -0.12  0.18                  
          question3      180.90  13.450    0.02 -0.09  0.04            
          question4      319.86  17.885   -0.06  0.11  0.06  0.02      
          modalityimage  108.09  10.397   -0.06  0.21  0.07  0.0

Warning message:
In commonArgs(par, fn, control, environment()) :
  maxfun < 10 * length(par)^2 is not recommended.


The model does a pretty good job of estimating variance for `modality`. We als got a warning: "maxfun < 10 * length(par)^2 is not recommended.". 

# Final model

In [34]:
re_formula = Formula("(1 + question + modality | subject) + (1 + question + modality | item)")
DG.fit_transform({'corr.item':np.eye(6), 'sd.re_formula':str(re_formula)}, seed=2025)
# convert to Pandas DataFrame
df = DG.to_pandas()

In [36]:
R(fmt_script(df=df, shared_re=re_formula))

Warning message:
In commonArgs(par, fn, control, environment()) :
  maxfun < 10 * length(par)^2 is not recommended.



 SHARED model summary
Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  method [lmerModLmerTest]
Formula: rt ~ modality + question + (1 + question + modality | subject) +  
    (1 + question + modality | item)
   Data: df
Control: lmerControl(optimizer = "bobyqa")

      AIC       BIC    logLik  deviance  df.resid 
 643435.1  643876.2 -321668.6  643337.1     59951 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.3384 -0.6667  0.0036  0.6694  4.6004 

Random effects:
 Groups   Name        Variance Std.Dev. Corr                         
 subject  (Intercept)  435.22  20.862                                
          question1    130.99  11.445   -0.03                        
          question2    118.53  10.887   -0.11  0.20                  
          question3    195.16  13.970    0.02  0.04  0.20            
          question4    317.72  17.825   -0.04  0.06 -0.03  0.11      
          modality1    111.99  10.582    0.19  0.17 -0.02  0.04 -

Warning message:
In commonArgs(par, fn, control, environment()) :
  maxfun < 10 * length(par)^2 is not recommended.


# Accurate versus practical

Here, we are using simulated data, and can alter parameters in such a way to achieve the results we want. Real data won't be so structured (an cannot be manipulated post hoc). Here, we show that a model(s) that align conceptually with our hypotheses work. Next steps are to settle on a final model to calculate power for variables like `subjects` and `items`. Once we have an idea about how many items we might need, we can run some pilot subjects and extract variance estimates from those data to build a better power analysis. 